In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

import tensorflow as tf
#These 2 lines are required in order to use GPU for neural networks
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Conv1D, Conv2D, Conv3D
from tensorflow.python.client import device_lib

C:\Users\junho\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\junho\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\junho\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Importing raw_signals, combined_features_df, and meta_data_df

In [2]:
#Load raw_signals in array format
raw_signals = np.load('../data/created_data_files/ptb_raw_signals.npz')
raw_signals = raw_signals['arr_0']

In [3]:
raw_signals.shape

(21837, 1000, 12)

In [4]:
created_features_df = pd.read_csv('../data/created_data_files/combined_features')

In [5]:
meta_data_df = pd.read_csv('../data/created_data_files/meta_data_df')

In [6]:
# Split data into train and test
test_fold = 10

# Train
X_train = raw_signals[np.where(meta_data_df.strat_fold != test_fold)]
y_train = meta_data_df[(meta_data_df.strat_fold != test_fold)].iloc[:,-5:]

# Test
X_test = raw_signals[np.where(meta_data_df.strat_fold == test_fold)]
y_test = meta_data_df[meta_data_df.strat_fold == test_fold].iloc[:,-5:]

We need to reshape X_train and X_test so that they can be used as input into the mtex-cnn architecture

In [7]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

In [8]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

Let's build a first pass fully-connected neural network just to see what would happen. Let's give it 64 outputs in the first layer, 32 in the second, before flattening the data to fit the results, and an output layer of 5 categories.

In [9]:
# #Initial model for testing
# model = Sequential()

# model.add(Dense(64, activation = 'relu', input_shape=(1000,12)))
# model.add(Dense(32, activation='relu'))
# model.add(Flatten())
# model.add(Dense(5, activation= 'softmax'))

# model.compile(loss = 'categorical_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])

# model.summary()

In [10]:
# #Initial model fitting
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=512)

The model ends with an accuracy of around 0.2-0.25. If the model were given a few more epochs, we would probably end up with an accuracy of around 4.3-4.5.

This result is probably because of flattening the data

Set up an MTEX-CNN.

In [11]:
k = raw_signals.shape[1]
n = raw_signals.shape[0]

In [12]:
raw_signals.shape

(21837, 1000, 12)

In [13]:
raw_signals = raw_signals.reshape(raw_signals.shape[0], raw_signals.shape[1], raw_signals.shape[2], 1)

In [14]:
raw_signals[0][999]

array([[ 0.022],
       [-0.031],
       [-0.054],
       [ 0.005],
       [ 0.038],
       [-0.042],
       [-0.001],
       [ 0.107],
       [-0.149],
       [ 0.143],
       [-0.035],
       [-0.12 ]])

In [15]:
print(k)

1000


In [23]:
mtex_cnn = Sequential()

'''
The first part of the model consists of a series of 2d Convolutional layers with 'half padding', which seems to be
the same as 'same' padding in keras. Kernel size should be the column size, while stride is explicitly defined
as 2x1
'''
mtex_cnn.add(Conv2D(12, activation='relu', kernel_size=(500,1), strides = (2,1), 
                    input_shape = (1000,12,1), padding='same'))
mtex_cnn.add(Conv2D(12, activation='relu', kernel_size=(500,1), strides = (2,1), padding='same'))

'''
Add the 1x1 convolutional layer that is at the end of the convolutional 2d layers
'''
mtex_cnn.add(Conv2D(12, activation='relu', kernel_size=(1,1)))


"""
The third part of the mtex-cnn architecture is a series of stacked 1D covolutional layers followed by 2 dense layers,
the second of which is the output layer of the network
"""
mtex_cnn.add(Conv1D(12, activation='relu', kernel_size=1, padding='same'))

"""
Flatten the output of the Conv1D filters
"""
mtex_cnn.add(Flatten())

"""
The final aspect of the model is 2 dense layers that takes the flattened 1D vector as input and is then connected to
an output softmax activated dense layer that assigns the probabilities of belonging to different classes
"""
mtex_cnn.add(Dense(12, activation = 'relu'))
mtex_cnn.add(Dense(5, activation = 'softmax'))

In [24]:
mtex_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 500, 12, 12)       6012      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 250, 12, 12)       72012     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 250, 12, 12)       156       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 250, 12, 12)       156       
_________________________________________________________________
flatten_1 (Flatten)          (None, 36000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                432012    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [25]:
mtex_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Check if Keras is running on GPU

In [26]:
print("Num of GPUs available: ", len(tf.test.gpu_device_name()))

Num of GPUs available:  13


In [29]:
with tf.device('/gpu:0'):
    mtex_cnn.fit(X_train, y_train, epochs=5, batch_size=512)

Epoch 1/5
39/39 [==============================] - 103s 1s/step - loss: 95866.4922 - acc: 0.2882
Epoch 2/5
39/39 [==============================] - 40s 1s/step - loss: 293707.1562 - acc: 0.2909
Epoch 3/5
39/39 [==============================] - 40s 1s/step - loss: 1017831.5625 - acc: 0.2897
Epoch 4/5
39/39 [==============================] - 40s 1s/step - loss: 2666671.2500 - acc: 0.3016
Epoch 5/5
39/39 [==============================] - 167s 4s/step - loss: 3638750.5000 - acc: 0.2970


## References

MTEX-CNN: Multivariate Time series EXplanations for Predictions with Convolutional Neural Networks